In [ ]:
import numpy as np

#### base

In [ ]:
class base_layer:

    def __init__(self) -> None:
        pass

    def forward(self):
        raise NotImplementedError("forward is not implemented")
    
    def backward(self):
        raise NotImplementedError("backward is not implemented")

#### conv

In [ ]:
class conv(base_layer):

    def __init__(self) -> None:
        super().__init__()

    def convolving(self):
        raise NotImplementedError("convolving is not implemented")

#### psr

In [ ]:
class psr_layer(base_layer):
    
    def __init__(self) -> None:
        super().__init__()

    def grader(self):
        raise NotImplementedError("grader is not implemented")
    
    # NOTE: کاست رو یطوری باید بنویسیم که نتیجه غایی رو بدیم بهش و کاست ر بده بهمون
    # کاست رو باید توی نتورک بیایم برای نود تعریف کنیم!
    def cost(self):
        raise NotImplementedError("cost is not implemented")
    
    # REST OF NET
    def ron(self):
        raise NotImplementedError("rest of net is not implemented")

#### qconv

In [ ]:
class qconv(conv):
    # TODO: add ACTIVATION, STRID and PAD
    # if you like DO: iylDO: add upsilon value
    def __init__(self, kernel: qml.QNode, kernel_size: int, weight, lr: float = 0.01) -> None:
        self.kernel = kernel
        self.kernel_size = kernel_size
        self.weight = weight
        # self.activator = activator
        # self.strid = strid
        # self.pad = pad
        self.lr = lr

    # NOTE: این صرفا یه ورودی می‌گیره، دیتاست ر شبکه می‌گیره
    def forward(
            self, input_data=None, # صرفا موقع فوروارد جدید مقدار داره
            input_theta=None, # صرفا موقع پارامتر شیقت رول مقدار داره
            input_label=None # صرفا موقع فوروارد جدید مقدار داره
                ):
        
        # NOTE: اگه دیتای ورودی جدیدی نداشته باشیم لست اینپوتو استفاده می‌کنه
        if isinstance(input_data, type(None)):
        # if input_data  == None:
            data = self.last_data
        # -NOTE: در غیر این‌صورت لست اینپوتو آپدیت می‌کنه و از ورودی جدید استفاده می‌کنه
        else:
            data = self.last_data = input_data

        if isinstance(input_label, type(None)):
            label = self.last_label
        else:
            label = self.last_label = input_label

        # NOTE: اگه تتای جدیدی نباشه از تتای درونی استفاده می‌کنه
        # if input_theta == None:
        if isinstance(input_theta, type(None)):
            theta = self.weight
        # -NOTE: در غیر این‌صورت از تتاهای جدید استفاده می‌کنه و <<البته>> تتای درونی رو آپدیت نمیکنه
        # چون آپدیت تتا یه پروسه کلا جداست
        else:
            theta = input_theta
        
        # WARN: این با یکمدار بعنوان کرنل چند کرنل رو اجرا می‌کنه

        xd, yd = data.shape

        mxd = xd-(self.kernel_size-1)
        myd = yd-(self.kernel_size-1)

        output = np.zeros((self.weight.shape[0], mxd, myd))
        
        for i, j in enumerate(theta):
            output[i] = self.convolving(data, j)

        output = np.sum(output, 0)

        return output.numpy()
    
    def convolving(self, data, theta):

        xd, yd = data.shape

        # WARNING: این فقط برای یک قدم یک قدم بدون پدینگ جلو رفتنه
        mxd = xd-(self.kernel_size-1)
        myd = yd-(self.kernel_size-1)
        map = np.zeros((mxd, myd))

        for row in range(mxd):
            for column in range(myd):
                # NOTE: یه تارگت برمی‌داریم
                target = data[row: row+self.kernel_size, column: column+self.kernel_size]
                res = self.kernel(
                                np.pi/(target.reshape(self.kernel_size*self.kernel_size)+1), 
                                theta
                                ).numpy()
                map[row, column] = res

        return map.numpy()
    
    # TODO: باید گرادیان حساب کنیم و وزنارو آپدیت کنیم
    def backward(self, *args):
        raise NotImplementedError("backward is not implemented")

In [ ]:
class qconv_psr(qconv, psr_layer): # qconv_layer with parameter shift rule

    def __init__(self, kernel: qml.QNode, kernel_size: int, weight, lr: float = 0.01) -> None:
        super().__init__(kernel, kernel_size, weight, lr)

        # TODO: باید گرادیان حساب کنیم و وزنارو آپدیت کنیم
    def backward(self, *args):
        # WARN: همچنان برای صرف همون فرمت کار میکنه
        self.gradient = np.zeros(self.weight.shape)

        for i, j in enumerate(self.weight):
            self.gradient[i] = self.grader(i, j)

        self.gradient = self.gradient.numpy()
        self.weight -= self.gradient*self.lr

        return args
    
    def grader(self, index, value):
        differ = np.zeros(self.weight.shape)
        differ[index] = 0.001*value # -> UPSILON * THETA

        up_weights = self.weight+differ
        down_weights = self.weight-differ

        up_cost = self.cost(
                        self.last_label, 
                        self.ron.res(self.forward(input_theta=up_weights))
                        )
        down_cost = self.cost(
                        self.last_label, 
                        self.ron.res(self.forward(input_theta=down_weights))
                        )
        
        return (up_cost-down_cost)/(2*0.001)

#### pooling

In [ ]:
class pooling2d(base_layer):

    def __init__(self) -> None:
        super().__init__()

    def forward(self, input_data):
        mxd = ((input_data.shape[0]-self.kernel_size)/2)+1
        myd = ((input_data.shape[1]-self.kernel_size)/2)+1
        
        map = np.zeros(mxd, myd)

        for row in range(mxd):
            for column in range(myd):
                target = input_data[row: row+self.kernel_size, column: column+self.kernel_size]
                res = self.pooling(target)
                map[row, column] = res

    def pooling(self):
        raise NotImplementedError("pooling is not implemented")
    
    def backward(self, *args):
        return args

In [ ]:
class max_pooling2d(pooling2d):
    # TODO: add STRID and PAD
    # WARN: این فثط برای استرید ۲ کار می‌کنه
    def __init__(self, size=2) -> None:
        self.kernel_size = size
        # self.activator = activator
        # self.strid = strid
        # self.pad = pad
    
    def pooling(t):
        return np.max(t)

#### dense

In [ ]:
class dense(base_layer):
    # TODO: add the feature that make initial weights by the way
    def __init__(self, in_dim, out_dim, in_weights, lr = 0.01) -> None:
        self.lr = lr
        self.weight = weights
        self.in_dim = in_dim
        self.out_dim = out_dim

In [ ]:
class classic_dense:

    def __init__(self, in_dim, out_dim, weights, lr = 0.01):
        self.lr = lr
        self.weight = weights
        self.in_dim = in_dim
        self.out_dim = out_dim

    def forward_pass(self, input_data, *args):
        self.last_data = input_data
        output = np.matmul(input_data, self.weight)
        return output

    
    def backprop(self, grad_previous):
        self.grad = np.matmul((self.X.transpose()), grad_previous)/self.in_dim
        self.weight = self.weight - (self.lr*self.grad)
        output = np.matmul(grad_previous, self.Theta.transpose())
        return output

In [ ]:
class classic_dense_psr:


    def __init__(self, in_dim, out_dim, weights, lr = 0.01):
        self.lr = lr
        self.weight = weights
        self.in_dim = in_dim
        self.out_dim = out_dim

    def forward_pass(
            self, input_data=None,
            input_theta=None,
            input_label=None):
        
        # NOTE: اگه دیتای ورودی جدیدی نداشته باشیم لست اینپوتو استفاده می‌کنه
        if isinstance(input_data, type(None)):
            data = self.last_data
        # -NOTE: در غیر این‌صورت لست اینپوتو آپدیت می‌کنه و از ورودی جدید استفاده می‌کنه
        else:
            data = self.last_data = input_data

        if isinstance(input_label, type(None)):
            label = self.last_label
        else:
            label = self.last_label = input_label

        # NOTE: اگه تتای جدیدی نباشه از تتای درونی استفاده می‌کنه
        if isinstance(input_theta, type(None)):
            theta = self.weight
        # -NOTE: در غیر این‌صورت از تتاهای جدید استفاده می‌کنه و <<البته>> تتای درونی رو آپدیت نمیکنه
        # چون آپدیت تتا یه پروسه کلا جداست
        else:
            theta = input_theta

        output = np.matmul(data, self.weight)
        return output
    

        # TODO: باید گرادیان حساب کنیم و وزنارو آپدیت کنیم
    def backward(self, *args):
        # WARN: همچنان برای صرف همون فرمت کار میکنه
        self.gradient = np.zeros(self.weight.shape)

        for i, j in enumerate(self.weight):
            self.gradient[i] = self.grader(i, j)

        self.gradient = self.gradient.numpy()
        self.weight -= self.gradient*self.lr

        return args
    
    def grader(self, index, value):
        differ = np.zeros(self.weight.shape)
        differ[index] = 0.001*value # -> UPSILON * THETA

        up_weights = self.weight+differ
        down_weights = self.weight-differ

        up_cost = self.cost(
                        self.last_label, 
                        self.ron.res(self.forward(input_theta=up_weights))
                        )
        down_cost = self.cost(
                        self.last_label, 
                        self.ron.res(self.forward(input_theta=down_weights))
                        )
        
        return (up_cost-down_cost)/(2*0.001) # > /2*upsilon

#### Net

In [ ]:
class Seq:

    def __init__(self, funcs):
        self.funcs = funcs

    def res(self, input):
        res = input
        for i in self.funcs:
            res = i.forward(res, res)
        return res

In [ ]:
class PSR_Net:
# WARN: صرفا برای مدارهایی که تمامشون با پارامتر شیقت رول باشه( یا از اول تا یجایی جتما باشه)

    # WARN: اینجا م دیتاست نمی‌گیریم و یدوه دیتا میگیریم. دیتاست رو توی ترین میگیریم
    def __init__(self, network, cost) -> None :
        self.network = network
        self.cost = cost
        self.predict()

    # WARN: این قضیه بهینه نیست و اگه لایه ای از پارامنر شیفت استفاده نکنه کلی کاست و آراوان اضافه ایجاد می‌کنه
    def predict(self):
        for i, j in enumerate(self.network):
            j.cost = self.cost
            j.ron = Seq(self.network[i+1:])

    # TODO: صحت و کاست و تایم و اینارم برگردون
    def train(self, dataset, labelset):
        for i, j in zip(dataset, labelset):
            self.forward(i, j)
            self.backward()

    def forward(self, data, label):
        res = data
        for i in self.network:
            res = i.forward(res, label)

        return res
    
    def backward(self):
        back_g = 0
        for i in (reversed(self.Network)):
            back_g = i.backward(back_g)

#### cost

In [ ]:
def cross_entropy(target, prediction):
    l = np.multiply(target, prediction)
    return -np.log(l.max()).numpy()

#### actv

In [ ]:
class actv(base_layer):

    def __init__(self) -> None:
        super().__init__()

    def actv_func(self):
        raise NotImplementedError("activation function is not implemented")
    
    def backward(self, *args):
        return args

In [ ]:
def sf(x):
    return np.power(np.e, x)

class soft_max_1d(actv):

    def __init__(self) -> None:
        super().__init__()

    def forward(self, input_data):
        return self.actv_func(input_data)

    def actv_func(self, data):
        a = list(map(sf, data))
        s = sum(a)
        res = []
        for i in a:
            res.append(i/s)
        return res
    
    def backward(self, *args):
        return args

In [ ]:
def sm(x):
    return (1/(1+np.power(np.e, -x)))

class sigmoid(actv):

    def __init__(self) -> None:
        super().__init__()

    def forward(self, input_data):
        return self.actv_func(input_data)

    def actv_func(self, data):
        return list(map(sm, data))
    
    def backward(self, *args):
        return args